#### 3.5 사용자의 평가경향을 고려한 CF
    CF의 정확도를 더 개선시키기 위해서는 사용자의 평가경향(user bias)을 고려해서 예측치를 조정하는 것이다.
    평가경향을 예로 들면, 평가를 전체적으로 낮게해서 최고점이 5점만점에 3점인 고객도 있는 반면 평가를 전체적으로 높게 해서 
    최저점이 3점인 고객도 있다.

#### 알고리즘 작동 원리
1. 각 사용자의 평점평균을 구함
2. 각 아이템의 평점을 각 사용자의 평균에서의 차이(평점-해당 사용자의 평점 평균)로 변환 - called 평점편차
3. 평점편차를 사용해서 해당 사용자의 해당 아이템의 편차 예측값을 구한다. 구체적으로는 해당 사용자의 이웃을 구하고  
    이들 이웃의 해당 아이템에 대한 평점편차와 유사도를 가중평균한다.
4. 현 사용자의 평균에 편차 예측값을 더해준다.
5. 예측값을 구할 수 없는 경우에 지금까지는 3.0을 할당했는데, 이번에는 해당 사용자의 평점평균으로 대체한다.

- 추천시스템_1에서 사용한 데이터 불러오기 및 rmse, 평점 matrix, 코사인 유사도 구하기

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 데이터 읽어 오기 
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('u.user', sep='|', names=u_cols, encoding='latin-1')
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 'unknown', 
          'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 
          'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
          'Thriller', 'War', 'Western']
movies = pd.read_csv('u.item', sep='|', names=i_cols, encoding='latin-1')
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('u.data', sep='\t', names=r_cols, encoding='latin-1')

# timestamp 제거 
ratings = ratings.drop('timestamp', axis=1)
# movie ID와 title 빼고 다른 데이터 제거
movies = movies[['movie_id', 'title']]

# train, test 데이터 분리
from sklearn.model_selection import train_test_split
x = ratings.copy()
y = ratings['user_id']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)

# 정확도(RMSE)를 계산하는 함수 
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

# 모델별 RMSE를 계산하는 함수 
def score(model, neighbor_size=0):
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])
    y_pred = np.array([model(user, movie, neighbor_size) for (user, movie) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

# train 데이터로 Full matrix 구하기 
rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')

# train set 사용자들의 Cosine similarities 계산
from sklearn.metrics.pairwise import cosine_similarity
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)

- train 데이터의 user의 rating 평균과 영화의 평점편차 계산 

In [2]:
# 유저별 평균
rating_mean = rating_matrix.mean(axis=1)
# 3.5파트에서 제일 중요한 부분으로 평점평균으로 각 점수를 상대화 한 것이다. 머신러닝에서의 scaler를 적용했다고 생각하면 된다.
rating_bias = (rating_matrix.T - rating_mean).T

In [3]:
rating_matrix

movie_id,1,2,3,4,5,6,7,8,9,10,...,1671,1672,1673,1675,1676,1677,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,2.0,NaN,NaN,4.0,5.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
rating_bias

movie_id,1,2,3,4,5,6,7,8,9,10,...,1671,1672,1673,1675,1676,1677,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,-0.568627,-0.568627,1.431373,0.431373,-2.568627,1.431373,-0.568627,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.382979,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,0.068702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.702703,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,-1.537500,NaN,NaN,0.462500,1.462500,-0.537500,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,0.764706,NaN,NaN,NaN,NaN,NaN,-0.235294,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
a1=scaler.fit_transform(rating_matrix.T)
rating_bias2=pd.DataFrame(a1.T,index=rating_matrix.index,columns=rating_matrix.columns)
rating_bias2

movie_id,1,2,3,4,5,6,7,8,9,10,...,1671,1672,1673,1675,1676,1677,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,-0.452573,-0.452573,1.139235,0.343331,-2.044381,1.139235,-0.452573,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.367270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,0.052494,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.735980,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,-1.501452,NaN,NaN,0.451656,1.428211,-0.524898,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,0.948122,NaN,NaN,NaN,NaN,NaN,-0.291730,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- 표준화스케일링은 참고를 위한 것이다. 

In [6]:
def CF_knn_bias(user_id,movie_id,neighbor_size=0):
    if movie_id in rating_bias:
        # 입력받은 유저 id 별 유사도 가져오기
        sim_scores = user_similarity[user_id].copy()
        # 현 movie의 평점편차 가져오기
        movie_ratings = rating_bias[movie_id].copy()
        # 현 movie에 대한 rating이 없는 사용자 삭제
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index
        movie_ratings = movie_ratings.drop(none_rating_idx)
        sim_scores = sim_scores.drop(none_rating_idx)
        # 이웃의 수가 지정되지 않은 경우
        if neighbor_size == 0:
            prediction = np.dop(sim_scores,movie_ratings) / sim_scores.sum()
            prediction = prediction + rating_mean[user_id]
        # 이웃의 수가 지정되는 경우
        else:
            if len(sim_scores) >1:
                neighbor_size = min(neighbor_size,len(sim_scores))
                sim_scores = np.array(sim_scores)
                movie_ratings = np.array(movie_ratings)
                user_idx = np.argsort(sim_scores)
                sim_scores = sim_scores[user_idx][-neighbor_size:]
                movie_ratings = movie_ratings[user_idx][-neighbor_size:]
                prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
                prediction = prediction + rating_mean[user_id]
            else:
                prediction = rating_mean[user_id]
    else:
        prediction = rating_mean[user_id]
    return prediction

In [7]:
def CF_knn_bias2(user_id,movie_id,neighbor_size=0):
    if movie_id in rating_bias2:
        # 입력받은 유저 id 별 유사도 가져오기
        sim_scores = user_similarity[user_id].copy()
        # 현 movie의 평점편차 가져오기
        movie_ratings = rating_bias2[movie_id].copy()
        # 현 movie에 대한 rating이 없는 사용자 삭제
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index
        movie_ratings = movie_ratings.drop(none_rating_idx)
        sim_scores = sim_scores.drop(none_rating_idx)
        # 이웃의 수가 지정되지 않은 경우
        if neighbor_size == 0:
            prediction = np.dop(sim_scores,movie_ratings) / sim_scores.sum()
            prediction = prediction + rating_mean[user_id]
        # 이웃의 수가 지정되는 경우
        else:
            if len(sim_scores) >1:
                neighbor_size = min(neighbor_size,len(sim_scores))
                sim_scores = np.array(sim_scores)
                movie_ratings = np.array(movie_ratings)
                user_idx = np.argsort(sim_scores)
                sim_scores = sim_scores[user_idx][-neighbor_size:]
                movie_ratings = movie_ratings[user_idx][-neighbor_size:]
                prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
                prediction = prediction + rating_mean[user_id]
            else:
                prediction = rating_mean[user_id]
    else:
        prediction = rating_mean[user_id]
    return prediction

- 사용자 평균 차감

In [8]:
score(CF_knn_bias, 36)

0.9406993919883266

- 표준화 스케일링

In [9]:
score(CF_knn_bias2, 36)

0.9402381217703766

기존의 이웃의 수가 36일때 rmse는 1.0152였는데 모델의 성능은 향상되었다.  
사용자 평균을 빼서 상대화 한 경우보다 표준화스케일링을 사용자 기반으로 한 경우가 더 rmse가 작지만 큰 차이는 없다.

#### 사용자의 성향 기반 추천 시스템 구현

In [10]:
movies=movies.set_index('movie_id')

In [11]:
def recom_movie(user_id,n_items,neighbor_size=30):
    user_movie= rating_bias.loc[user_id].copy()
    for movie_id in rating_bias:
        if pd.notnull(user_movie.loc[movie_id]):
            user_movie.loc[movie_id] = 0
        else:
            user_movie.loc[movie_id] = CF_knn_bias(user_id,movie_id,neighbor_size)
        movie_sort= user_movie.sort_values(ascending=False)[:n_items]
        recom_movies = movies.loc[movie_sort.index]
        recommendations = recom_movies['title']
        return recommendations
    

In [12]:
recom_movie(user_id=2,n_items=5,neighbor_size=36)

movie_id
285            Secrets & Lies (1996)
50                  Star Wars (1977)
127            Godfather, The (1972)
311    Wings of the Dove, The (1997)
251           Shall We Dance? (1996)
Name: title, dtype: object

- 사용자 성향 기반으로 추천 시스템을 만들면 유저 2의 경우 5개의 다음의 영화를 추천할 수 있다.

#### 3.6 신뢰도 기반 사용자 필터링을 통한 이웃 조정 후 CF 
    사용자별 공통 평가 수 계산

In [13]:
rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')
rating_binary1= np.array(rating_matrix>0).astype('float')
rating_binary2 = rating_binary1.T
counts = np.dot(rating_binary1,rating_binary2)
counts = pd.DataFrame(counts,index=rating_matrix.index, columns=rating_matrix.index).fillna(0)

In [14]:
def CF_knn_bias_sig(user_id,movie_id,neighbor_size=0):
    if movie_id in rating_bias:
        # 현 user와 다른 사용자 간의 유사도 가져오기
        sim_scores = user_similarity[user_id]
        # 현 movie의 평점편차 가져오기
        movie_ratings = rating_bias[movie_id]
        # 현 movie에 대한 rating이 없는 사용자 표시
        no_rating = movie_ratings.isnull()
        # 현 사용자와 다른 사용자간 공통 평가 아이템 수 가져오기 
        common_counts = counts[user_id]
        # 공통으로 평가한 영화의 수가 SIG_LEVEL보다 낮은 사용자 표시
        low_significance = common_counts < SIG_LEVEL
        # 평가를 안 하였거나, SIG_LEVEL이 기준 이하인 user 제거
        none_rating_idx = movie_ratings[no_rating | low_significance].index
        movie_ratings = movie_ratings.drop(none_rating_idx)
        sim_scores = sim_scores.drop(none_rating_idx)
        
##### (2) Neighbor size가 지정되지 않은 경우        
        if neighbor_size == 0:
            # 편차로 예측값(편차 예측값) 계산
            prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
            # 편차 예측값에 현 사용자의 평균 더하기
            prediction = prediction + rating_mean[user_id]
                
##### (3) Neighbor size가 지정된 경우            
        else:
            # 해당 영화를 평가한 사용자가 최소 MIN_RATINGS 이상인 경우에만 계산            
            if len(sim_scores) > MIN_RATINGS:
                # 지정된 neighbor size 값과 해당 영화를 평가한 총사용자 수 중 작은 것으로 결정
                neighbor_size = min(neighbor_size, len(sim_scores))
                # array로 바꾸기 (argsort를 사용하기 위함)
                sim_scores = np.array(sim_scores)
                movie_ratings = np.array(movie_ratings)
                # 유사도를 순서대로 정렬
                user_idx = np.argsort(sim_scores)
                # 유사도와 rating을 neighbor size만큼 받기
                sim_scores = sim_scores[user_idx][-neighbor_size:]
                movie_ratings = movie_ratings[user_idx][-neighbor_size:]
                # 편차로 예측치 계산
                prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
                # 예측값에 현 사용자의 평균 더하기
                prediction = prediction + rating_mean[user_id]
            else:
                prediction = rating_mean[user_id]
    else:
        prediction = rating_mean[user_id]
    if prediction > 5:
        prediction = 5
    elif prediction < 1:
        prediction = 1

    return prediction    

In [15]:
SIG_LEVEL = 3
MIN_RATINGS = 2
score(CF_knn_bias_sig, 36)

0.9389661549758815

In [16]:
def recom_movie2(user_id,n_items,neighbor_size=30):
    user_movie= rating_bias.loc[user_id].copy()
    for movie_id in rating_bias:
        if pd.notnull(user_movie.loc[movie_id]):
            user_movie.loc[movie_id] = 0
        else:
            user_movie.loc[movie_id] = CF_knn_bias_sig(user_id,movie_id,neighbor_size)
        movie_sort= user_movie.sort_values(ascending=False)[:n_items]
        recom_movies = movies.loc[movie_sort.index]
        recommendations = recom_movies['title']
        return recommendations

In [17]:
recom_movie2(user_id=2,n_items=5,neighbor_size=36)

movie_id
285            Secrets & Lies (1996)
50                  Star Wars (1977)
127            Godfather, The (1972)
311    Wings of the Dove, The (1997)
251           Shall We Dance? (1996)
Name: title, dtype: object

#### 3.7 아이템 기반 CF(Item Based Collaborative Filtering)
    사용자 기반 CF는 데이터가 충분할 경우 정확한 추천이 가능하지만 느리고 자주 업데이트를 해야한다.
    반면, UBCF는 데이터가 적어도 추천이 가능하면 자주 업데이트를 하지 않아도 된다.
    그러나 개인화된 추천은 하기 어렵다.
    아마존과 같은 대규모 데이터를 다뤄야 하는 상업용 사이트에서는 IBCF에 기반한 알고리즘이 사용되는 것으로 알려져 있다.
    아래에서는 IBCF를 구현하겠다.다만, 이웃의 크기를 고려하지 않은 UBCF를 수정하겠다.

In [18]:
# train set의 모든 가능한 아이템 pair의 Cosine similarities 계산
rating_matrix_t = np.transpose(rating_matrix)
matrix_dummy = rating_matrix_t.copy().fillna(0)
item_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
item_similarity = pd.DataFrame(item_similarity, index=rating_matrix_t.index, columns=rating_matrix_t.index)

- 주어진 영화의 (movie_id) 가중평균 rating을 계산하는 함수, 
- 가중치는 주어진 아이템과 다른 아이템 간의 유사도(item_similarity)

In [19]:
def CF_IBCF(user_id, movie_id):
    if movie_id in item_similarity:      # 현재 영화가 영화별 유사도에 있는지 확인
        # 현재 영화와 다른 영화의 similarity 값 가져오기
        sim_scores = item_similarity[movie_id]
        # 현 사용자의 모든 rating 값 가져오기
        user_rating = rating_matrix_t[user_id]
        # 사용자가 평가하지 않은 영화 index 가져오기
        non_rating_idx = user_rating[user_rating.isnull()].index
        # 사용자가 평가하지 않은 영화 제거
        user_rating = user_rating.dropna()
        # 사용자가 평가하지 않은 영화의 similarity 값 제거
        sim_scores = sim_scores.drop(non_rating_idx)
        # 현 영화에 대한 예상 rating 계산, 가중치는 현 영화와 사용자가 평가한 영화의 유사도
        mean_rating = np.dot(sim_scores, user_rating) / sim_scores.sum()
    else:
        mean_rating = 3.0
    return mean_rating

변수가 두개만 들어가므로 SCORE함수를 재설정

In [20]:
def score(model):
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])
    y_pred = np.array([model(user, movie) for (user, movie) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

In [21]:
score(CF_IBCF)

1.0087356916609291

#### 3.8 추천 시스템의 성과측정지표

이진값에서의 성과측정지표
rmse나 mse는 연속값에서의 성과측정지표로 현업에서 구매했는지 안했는지의 여부처럼 이진 값에는 사용할 수 없다.  
이경우 accuracy,precision,recall,F1 score 등이 사용된다.  
그러나 데이터에서 True Nagtive 데이터가 일반적으로 다른 데이터보다 너무 많아 지표들이 0이나 1로 수렴해서 정확한 측정이 어렵다.  
그러므로 성과측정지표로 TN을 안쓰는 TPR을 사용하는 경우가 많다.  
TruePositiveRate이란 TP/(TP+FN) 이다.  